In [42]:
import tensorflow as tf
import autograd
from autograd import numpy as np
import numpy as anp
import time

import examples_lib
from examples_lib import get_logistic_ll_vec, get_logistic_ll

In [28]:

######################
# Logistic

def get_logistic_ll_vec(y, x_mat, theta):
    z = tf.tensordot(x_mat, theta, axes=1)
    return y * z + tf.math.log1p(z)

def get_logistic_ll(y, x_mat, theta):
    return tf.reduce_sum(get_logistic_ll_vec(y, x_mat, theta))


def np_get_logistic_ll_vec(y, x_mat, theta):
    z = x_mat @ theta
    return y * z + np.log1p(z)

def np_get_logistic_ll(y, x_mat, theta):
    return np.sum(np_get_logistic_ll_vec(y, x_mat, theta))


In [27]:
theta, x_mat, y = examples_lib.get_logistic_params(1000, 10)

In [32]:
assert np.abs(np_get_logistic_ll(y, x_mat, theta) -
              get_logistic_ll(y, x_mat, theta)) < 1e-12
assert np.max(np.abs(
    np_get_logistic_ll_vec(y, x_mat, theta) -
    get_logistic_ll_vec(y, x_mat, theta))) < 1e-12

In [30]:
tf.constant(theta)

<tf.Tensor: shape=(10,), dtype=float64, numpy=
array([0.00714032, 0.00786163, 0.00490904, 0.00031338, 0.00356672,
       0.00936083, 0.00938277, 0.00769708, 0.00887007, 0.00932182])>

In [41]:
np_get_logistic_ll_grad = autograd.grad(lambda theta: np_get_logistic_ll(y, x_mat, theta))
np_grad = np_get_logistic_ll_grad(theta)

theta_tf = tf.Variable(theta)
with tf.GradientTape() as tape:
    ll = get_logistic_ll(tf.constant(y), tf.constant(x_mat), theta_tf)
tf_grad = tape.gradient(ll, theta_tf)

assert np.max(np.abs(np_grad - tf_grad)) < 1e-12